<a href="https://colab.research.google.com/github/mansivyas26/perfusion_dataPipeline/blob/main/variables_diff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [454]:
import pandas as pd
import numpy as np
import random
import ast
import warnings
warnings.filterwarnings('ignore')

In [469]:
#read individual runs file .pkl
perfusion_data = pd.read_pickle("/content/drive/MyDrive/Steve_BEXT84_01-17-2024_13-56-14_all_data.pkl")

In [468]:
#read variable dictionary
variable_dict = pd.read_csv("/content/drive/MyDrive/Pefusion_DataPipeline/test_262.02 - test.csv")

variable_dict['possible_variables'] = variable_dict['possible_variables'].apply(ast.literal_eval)
#variable_dict.to_csv("/content/drive/MyDrive/Pefusion_DataPipeline/test.csv")


In [487]:
#reading the existing schema
schema = pd.read_csv("/content/drive/MyDrive/Pefusion_DataPipeline/m12_full_join.csv",nrows=10)

In [488]:
perfusion_data_subset = perfusion_data.head(3)

In [486]:
#check scehma with variable dictionary and renames the column if needed to standard_variable
def check_schema(schema,variable_dict):
  columns = schema.columns.tolist()
  for column in columns:
        match_found = False
        for idx, row in variable_dict.iterrows():
            if column in row['possible_variables']:
                match_found = True
                if column != row['standard_variable']:
                    print("columns that were renamed",column)
                    schema.rename(columns={column: row['standard_variable']}, inplace=True)

In [473]:
#function call
check_schema(schema,variable_dict)

columns that were renamed glucose_control_mode
columns that were renamed syringe_pump_manual_setrate
columns that were renamed time
columns that were renamed probe_ph_mv_measured
columns that were renamed glucose_pump_rate_setpoint


Compare .pkl file , variable_dicitionary and schema
Case A : if column in pkl file match one of the values in variable_dictionary['possible_variable']
Case 1 : if column in .pkl , variable dictionary['standard_variable'] and schema match
Case 2 : If column name is different ,in this case column is match with possible variables values and check if it's existing column if yes then column name is renamed to standard variable
Case 3 : If column name is new and does not exist in schema , in this case column is added to schema and variable_status is changed to exisitng **change schema df and schema table in bucket and big query,update variable_dictionary**
Case 4 : If column is in "Removed" stage
        Case 4.a both column exsist in schema and .pkl , removed change the name to standard variable
        Case 4.b column exist in schema but not in pkl,removed -> pkl['standard_variable'] = None
        Case 4.c


In [474]:

def check_columns(df):
  val = 'None'
  if val in df.columns:
      # Drop the column named 'None'
      df = df.drop(columns=[val])


In [475]:
check_columns(perfusion_data_subset)

In [476]:
check_columns(schema)

In [477]:
#compare .pkl file , variable dicitionary and Schema

def check_runs_with_variable_dict_new(pkl_df, variable_dict, schema_subset):

    removed_columns = []  # List to keep track of REMOVED columns
    list_of_files_ignored_in_pkl = ['timestamp','glucose_syringe_pump_buzzer']
    list_of_files_ignored_in_schema = ['resistance_qc_solenoid_flag','perfusion_date','perfusion_id','control_time','glucose_syringe_pump_dispensed_control', 'glucose_syringe_pump_buzzer_control']

    temporary_none = ['dialysis_compensate_scale_weight','dialysis_flux_valve_position','viscometer_temperature','dialysis_post_pressure','dialysis_compensate_pump_rate','dialysis_compensate_scale_flux','dialysis_pre_pressure','dialysis_pre_pressure_setpoint','dialysis_pressure_valve_position','dialysis_waste_scale_flux','dialysis_waste_scale_weight','']
    #list of variables to be added in schema
    for idx, row in variable_dict.iterrows():
            if row['standard_variable'] in list_of_files_ignored_in_schema:
                  pkl_df[row['standard_variable']] = "to be added"
    columns = pkl_df.columns.tolist()
    for column in columns:
        match_found = False
        for idx, row in variable_dict.iterrows():
           if column in row['possible_variables']:
                match_found = True
                if row['variable_status'] == "EXISTING" and column != row['standard_variable']:
                    pkl_df.rename(columns={column: row['standard_variable']}, inplace=True)
                    if column in schema_subset.columns:
                        schema_subset.rename(columns={column: row['standard_variable']}, inplace=True)
                elif row['variable_status'] == "NEW":
                    #add a column with name row['standard_variable'] and  value = None
                    variable_dict.loc[idx,'variable_status'] = "EXISTING"
                  #rename to standard variable
                    if row['standard_variable'] not in schema_subset.columns:
                        schema_subset[row['standard_variable']] = None

                elif row['variable_status'] == "REMOVED":
                   # print(f"Removed column identified: {column}")
                    removed_columns.append(column)
                    for c in removed_columns:
                      print(removed_columns)
                      pkl_df[c] = None


           if not match_found:
              if row['variable_status'] == "REMOVED":
               pkl_df[column] = "removed"
              for val in temporary_none:
                if val in schema_subset.columns and val not in pkl_df.columns:
        # Add a new column to the pkl DataFrame with the value initialized as None
                 pkl_df[val] = "None"


    cols_to_drop = list(set(pkl_df.columns).intersection(set(list_of_files_ignored_in_pkl)))
    print(cols_to_drop)
    pkl_df = pkl_df.drop(cols_to_drop,inplace=True,axis=1)








In [480]:
def check_columns_match(df1, df2):
    columns_df1 = set(df1.columns)
    columns_df2 = set(df2.columns)

    if columns_df1 != columns_df2:
        columns_only_in_df1 = columns_df1 - columns_df2
        columns_only_in_df2 = columns_df2 - columns_df1

        print("Columns only in .pkl file:", columns_only_in_df1)
        print("Columns only in schema:", columns_only_in_df2)
        print(len(columns_only_in_df2),"only_schema")
        print(len(columns_only_in_df1),"only_pkl")

        return False
    else:
        print(len(df1.columns),"pkl")
        print(len(df2.columns),"schema")
        print("Columns match between the two DataFrames.")
        return True

def append_df_row_wise(pkl_df, schema_df):
    #pkl_df = pkl_df.drop(columns=['None'],inplace=True)

    # Check if columns match
    if not check_columns_match(pkl_df, schema_df):
      return None
    else:
      schema_df = schema_df[sorted(schema_df.columns)]
      pkl_df = pkl_df[sorted(pkl_df.columns)]
      #print(pkl_df)

      """schema_df.reset_index(drop=True, inplace=True)
      schema_duplicates = schema_df.index.duplicated()
      print("schema",schema_df[schema_duplicates])
      pkl_df.reset_index(drop=True,inplace=True)"""

      combined_df = schema_df.append(pkl_df,ignore_index=True)
      print(combined_df)


In [483]:
print(append_df_row_wise(perfusion_data_subset,schema))

132 pkl
132 schema
Columns match between the two DataFrames.
    air_supply_pressure  arduino_reset_relay  auto_sampler_flag  \
0                   NaN                  NaN                NaN   
1                   NaN                  NaN                NaN   
2                   NaN                  NaN                NaN   
3                   NaN                  NaN                NaN   
4                   NaN                  NaN                NaN   
5                   NaN                  NaN                NaN   
6                   NaN                  NaN                NaN   
7                   NaN                  NaN                NaN   
8                   NaN                  NaN                NaN   
9                   NaN                  NaN                NaN   
10                  NaN                  NaN                NaN   
11                336.0                  0.0               -1.0   
12                339.0                  0.0                1.0   



In [482]:
print(check_runs_with_variable_dict_new(perfusion_data_subset,variable_dict,schema))

['control_time']
['control_time', 'glucose_syringe_pump_buzzer_control']
['control_time', 'glucose_syringe_pump_buzzer_control']
[]
None


In [452]:
from collections import Counter

my_list = perfusion_data_subset.columns.to_list()  # Example list

# Count occurrences of each element
counts = Counter(my_list)

# Find duplicate values
duplicates = [item for item, count in counts.items() if count > 1]

print("Duplicate values:", duplicates)

Duplicate values: []


In [ ]:
perfusion_data_subset['glucose_syringe_pump_volume_setpoint']

In [412]:
def duplicate_columns_func(df):
  duplicate_columns = df.columns[df.columns.duplicated()]
  print(duplicate_columns)

  # Drop just one duplicate column
  if duplicate_columns.any():
      column_to_drop = duplicate_columns[0]  # Select the first duplicate column to drop
      df.drop(columns=column_to_drop, inplace=True)
      print("Duplicate column dropped:", column_to_drop)
  else:
      print("No duplicate columns found.")

  print("DataFrame after dropping duplicate column:")
  print(df)

In [458]:
list1 = ['glucose_syringe_pump_volume_setpoint', 'syringe_pump_volume_setpoint', 'glucose_pump_rate_automatic_setpoint']
list2 = schema.columns.to_list()
list3 = perfusion_data_subset.columns.to_list()

In [459]:
def find_matching_values(list1, list2):
    return list(set(list1) & set(list2))

In [467]:
find_matching_values(list1,list3)
#find_matching_values(glucose_syringe_pump_volume_setpoint,glucose_syringe_pump_hour_setrate)

['syringe_pump_volume_setpoint', 'glucose_pump_rate_automatic_setpoint']

In [462]:
glucose_syringe_pump_volume_setpoint = ['syringe_pump_manual_setrate', 'glucose_pump_rate_manual_setpoint','glucose_pump_rate_setpoint', 'glucose_pump_rate', 'syringe_pump_rate','glucose_pump_rate_setpoint']
glucose_syringe_pump_hour_setrate = ['glucose_syringe_pump_hour_setrate', 'syringe_pump_manual_setrate', 'glucose_pump_rate_manual_setpoint','glucose_pump_rate_setpoint', 'glucose_pump_rate', 'syringe_pump_rate','glucose_pump_rate_setpoint']